# Tutorial: Periodically Driven Materials With Floquet Theory 

In [ ]:
using Plots
using Revise
using LatticeQM #The Floquet functionality is included in the Floquet module of LatticeQM

The Floquet module allows for simulating lattices which are externally driven with a periodic potential. This can change things like their band structure and topological phase. We start by opening a gap in graphene by driving it with circularly polarized light as discussed in PHYSICAL REVIEW B 79, 081406(R) (2009).

In order to apply a periodic drive to a static Hamiltonian, we first need to define it using the type `Floquet.periodicDrive` by calling `Floquet.periodicDrive(omega)`, where `omega` is the angular frequency of the drive.

In [ ]:
Floquet.periodicDrive(1)

As can be seen from the print-out, the `periodicDrive` does not do anything now. To change this, we add the Fourier components of our periodic potential using the `addFreq!` command. These are operators of the same dimension as the static Bloch Hamiltonian which we want to drive. For lattice momenta close to the K-point of graphene we can for example approximate circularly polarized light as the simple potential
\begin{equation}
    V(t) = 
    \begin{pmatrix}
        0 & \tau_z\cdot A \\
        0 & 0
    \end{pmatrix} \exp(-\tau_z \cdot i\omega t) +
    \begin{pmatrix}
        0 & 0 \\
        \tau_z\cdot A & 0
    \end{pmatrix} \exp(\tau_z \cdot i\omega t) \ ,
\end{equation}
where $\tau_z= \pm 1$ labels the chirality (ie K and K' point in graphene), $A$ is the amplitude of the gauge field, and $\omega$ is the frequency of the light. Let us stick to the case $\tau_z=1$. We realize that the non-zero Fourier components are
\begin{equation}
    c_{-1} =
    \begin{pmatrix}
        0 & 0 \\
        A & 0
    \end{pmatrix} \quad \text{and} \quad
    c_{1} =
    \begin{pmatrix}
        0 & A \\
        0 & 0
    \end{pmatrix} \ .
\end{equation}
We therefore add this information to our drive in the following way:

In [ ]:
function laserDrive(A::Number, omega::Number)
    d = Floquet.periodicDrive(omega)
    Floquet.addFreq!(d, [0 A; 0 0], 1)
    Floquet.addFreq!(d, [0 0; A 0], -1)
    return d
end

We already wrapped our drive in a function, such that we can easily vary A and omega later. Alternatively, we could define all this in one go by calling `Floquet.periodicDrive(omega, [[0 0;A 0],[0 A;0 0]], [-1,1])`. This yields the same outcome:

In [ ]:
laserDrive(2,50)

In [ ]:
Floquet.periodicDrive(50, [[0 0;2 0],[0 2;0 0]], [-1,1])

In order to apply this drive to a graphene lattice, we need to define it first, and then simply add the drive as additional argument in the `Spectrum.getbands` function.

In [ ]:
#initialize graphene lattice and corresponding path
lat = Geometries.honeycomb()
H = Operators.graphene(lat)
ks = kpath(lat; num_points=300)

In [ ]:
HF = Floquet.FloquetOperator(H, laserDrive(2,50), 50)

bands = Spectrum.getbands(HF, ks)
Floquet.keepfirstFBZ!(bands,HF)

The additional parameter `M` above decides how much of the Floquet-Hamiltonian is taken into account (see eg [Floquet Engineer's Handbook](https://arxiv.org/abs/2003.08252) on arXive for the details of the procedure). Generally, solving a $d$-dimensional problem with cut-off parameter $M$ requires the diagonalizing of a matrix of dimension $d\cdot(2M+1)$. The accuracy of the method is better for higher frequencies and lower driving amplitudes.

The `BandData` object which is returned is the same as for any time-independent Bloch Hamiltonian. We can for example easily plot it, which reveals the opening of a band gap at the K-point:

In [ ]:
N = round(Int, size(HF(ks.points[:,1]),1)/2)
cherns = Spectrum.getcherns(HF, 20, 20, N-1:N+2)

In [ ]:
p=plot(bands, dpi=100)
#savefig(p, "graphene_bands.png") #use this to save the Figure

### Topology

We also want to investigate how external driving can change the topology of a system. The above model is not accurate enough for this since the driving potential is a mere approximation, and the obtained result is only valid close to the K-point.

A simple toy-model which can be used to see topological effects is outlined in PHYSICAL REVIEW X 3, 031005 (2013). It consists of a Hamiltonian of the form
\begin{align}
    H_0(\vec{k}) &= \vec{d}(\vec{k})\cdot \vec{\sigma} \ , \\
    \text{with} \quad d_x(\vec{k}) &= a\sin(k_x) \ ,\\
    d_y(\vec{k}) &= a\cos(k_y) \ , \\
    d_z(\vec{k}) &= (\mu-J) -2b(2-\cos(k_x)-\cos(k_y)) + J\cos(k_x)\cos(k_y) \ ,
\end{align}
which is driven by a potential
\begin{equation}
    \Delta(t) = \Delta_0 \cos(\omega t) \sigma_z \ .
\end{equation}

We first implement the time-independent part:

In [ ]:
using LinearAlgebra: dot
using LatticeQM.Utils: σs

In [ ]:
#parameters
mu = 1
a = 4
b = 1.5
J = 1.5
#for mu,b,J>0 and mu/b<4 the system is topological (see PHYSICAL REVIEW X 3, 031005 (2013))

#Bloch Hamiltonian
function HB(k; mu=mu, a=a, b=b, J=J)
    d1 = a*sin(2*pi*k[1])
    d2 = a*sin(2*pi*k[2])
    d3 = (mu-J) - 2*b*(2 - cos(2*pi*k[1])- cos(2*pi*k[2])) + J*cos(2*pi*k[1])*cos(2*pi*k[2])
    ds = [d1, d2, d3]
    return sum([ds[i].*σs[i] for i=1:3])
end

We solve the static Hamiltonian and realize that the band structure is topological by calculating the Chern numbers of the bands. We can do this using the command `Spectrum.getcherns`.

In [ ]:
sqlat = Geometries.square()
ks2 = kpath(sqlat; num_points=400)

bands2 = Spectrum.getbands(HB, ks2)

In [ ]:
plot(bands2)

In [ ]:
cherns = Spectrum.getcherns(HB, 15, 15) #calculate Chern numbers
println("Chern number of the lower band: "*string(cherns[1]))
println("Chern number of the upper band: "*string(cherns[2]))

Interestingly, we can change these Chern numbers to zero by applying the drive. However, the model is still topologically non-trivial. This is why in the driven case, one should always use the so called _winding number_ (see PHYSICAL REVIEW X 3, 031005 (2013)) to be on the save side. We are demonstrating this here.

First, we define the drive:

Plotting the band structure, we do not see any big change:

In [ ]:
#parameters
ω = 14
D = 1.
M=5 # truncation parameter

drive = Floquet.periodicDrive(ω, [[D 0; 0 -D], [D 0; 0 -D]], [1,-1])
HF = Floquet.FloquetOperator(HB, drive, M)

bands3 = getbands(HF, ks2)

# Show first Floquet Brillouin zone
plot(bands3; ylims=(-ω/2,ω/2))

To see that the Chern numbers have changed, we can again use the command `Spectrum.getcherns` but with additional arguments for the drive. We can also use the `Spectrum.wavefunction` command to obtain wavefunctions of driven systems.

There is however one key difference here. The wavefunctions which are returned are only eigenfunctions of the Floquet-operator, which governs the time evolution over one full period, and are therefore not identical to the eigenfunctions of the time-evolution operator.

Additionally, the functions are returned in Fourier-space per default. To change this, one could set the `timespace=true`, but this is not necessary here. (Note that the eigenfunctions of the Floquet-operator are time-dependent. `wavefunctions(...;timespace=true)` returns them at time $0$.) Since the wavefunctions are in Fourier space, they also contain the information of all the Floquet bands, also the ones outside the first Floquet Brillouin zone (first FBZ). This means that we have to choose the right bands that correspond to the ones in the plot above (ie the ones in the first FBZ).

The frist FBZ lies in the middle of the spectrum. For a $d$-dimensional Bloch Hamiltonian the relevant bands are the ones with indices $2M+1$ to $2M+d$, where $M$ is the cut-off parameter. So in our case the lower band has index $2M+1$. We can get this Chern number alone using `Spectrum.getcherns(...,[2*M+1])`.

The calculations below show clearly that the drive has changed the Chern numbers of the bands to zero. However, they are still topologically non-trivial, as can be seen in the non-zero winding number.

In [ ]:
chern = Spectrum.getcherns(HF, 15, 15, [2*M+1]) #calculate "normal" chern number, will be zeros
windnum = Spectrum.getwindnum(HF, 15, 15, 2*M+1) #calculate winding number, will be -1
println("Chern number of the lower band with drive: "*string(chern[1]))
println("Winding Number of the lower band with drive:"*string(windnum))